# TechNova Partners - Analyse du Churn RH

**Projet :** Identification des causes de démission et modélisation prédictive  
**Client :** TechNova Partners (ESN spécialisée en transformation digitale)

---

## Contexte du Projet

TechNova Partners fait face à un turnover élevé. L'objectif est de :

1. **Analyser** les données RH pour identifier les différences entre employés partis et restés
2. **Construire** un modèle de classification pour prédire les démissions
3. **Extraire** les causes potentielles via l'interprétation du modèle (SHAP)

**Sources de données :**

- `data/extrait_sirh.csv` - Informations RH (âge, salaire, poste, ancienneté...)
- `data/extrait_eval.csv` - Évaluations de performance
- `data/extrait_sondage.csv` - Sondage employés + **variable cible**

---

## Structure du Notebook

**Partie 1 : Exploration des Données**

- Chargement et compréhension des fichiers
- Fusion et création du dataset central
- Analyse exploratoire et visualisations

**Partie 2 : Feature Engineering**

- Préparation des features (X)
- Encodage des variables catégorielles
- Gestion des corrélations

**Partie 3 : Modélisation Baseline**

- Modèle Dummy (référence)
- Modèle linéaire
- Modèle non-linéaire (arbre)

**Partie 4 : Gestion du Déséquilibre**

- Stratification
- Class weights / Undersampling / Oversampling (SMOTE)
- Validation croisée stratifiée

**Partie 5 : Optimisation et Interpretation**

- Fine-tuning des hyperparamètres (GridSearchCV)
- Feature importance globale (SHAP Beeswarm)
- Feature importance locale (SHAP Waterfall)

---


---

## 1. Importation des librairies


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.float_format", "{:.2f}".format)

plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("husl")

warnings.filterwarnings("ignore")

---

## 2. Chargement des données

Chargement des 3 fichiers CSV et examen de structure.


In [ ]:
df_sirh = pd.read_csv("data/extrait_sirh.csv")
df_eval = pd.read_csv("data/extrait_eval.csv")
df_sondage = pd.read_csv("data/extrait_sondage.csv")

print(f"Fichier SIRH : {df_sirh.shape[0]} lignes, {df_sirh.shape[1]} colonnes")
print(f"Fichier Évaluations : {df_eval.shape[0]} lignes, {df_eval.shape[1]} colonnes")
print(f"Fichier Sondage : {df_sondage.shape[0]} lignes, {df_sondage.shape[1]} colonnes")

---

## 3. Exploration initiale de chaque fichier

Avant de fusionner, comprenons le contenu et la structure de chaque fichier.


### 3.1 Fichier SIRH (extrait_sirh.csv)


#### Aperçu des premières lignes

Visualisons les premières lignes pour comprendre la structure et le contenu.


In [ ]:
df_sirh.head()

#### Structure et types de données

Analysons les types de colonnes, la mémoire utilisée et les valeurs non-nulles.


In [ ]:
df_sirh.info()

#### Statistiques descriptives

Calculons les statistiques de base (moyenne, écart-type, min, max, quartiles) pour les variables numériques.


In [ ]:
df_sirh.describe()

#### Analyse des variables catégorielles

Examinons les valeurs uniques et leur fréquence pour chaque variable catégorielle.


In [ ]:
print("Valeurs uniques des colonnes catégorielles SIRH :")
for col in df_sirh.select_dtypes(include="object").columns:
    print(f"\n{col}: {df_sirh[col].nunique()} valeurs uniques")
    print(df_sirh[col].value_counts())

### 3.2 Fichier Évaluations (extrait_eval.csv)


#### Aperçu des premières lignes


In [ ]:
df_eval.head()

#### Structure et types de données


In [ ]:
df_eval.info()

#### Statistiques descriptives


In [ ]:
print("Statistiques descriptives Evaluations (variables numeriques) :")
df_eval.describe()

#### Analyse des variables catégorielles


In [ ]:
# Valeurs uniques des colonnes catégorielles
print("Valeurs uniques des colonnes catégorielles Evaluations :")
for col in df_eval.select_dtypes(include="object").columns:
    print(f"\n{col}: {df_eval[col].nunique()} valeurs uniques")
    print(df_eval[col].value_counts())

### 3.3 Fichier Sondage (extrait_sondage.csv)


#### Aperçu des premières lignes


In [ ]:
df_sondage.head()

#### Structure et types de données


In [ ]:
df_sondage.info()

#### Statistiques descriptives


In [ ]:
df_sondage.describe()

#### Analyse des variables catégorielles


In [ ]:
for col in df_sondage.select_dtypes(include="object").columns:
    print(f"\n{col}: {df_sondage[col].nunique()} valeurs uniques")
    print(df_sondage[col].value_counts())

---

## 4. Identification des clés de jointure

Pour fusionner les 3 fichiers, nous devons identifier les colonnes qui permettent de faire le lien entre eux.


#### Analyse des colonnes identifiantes

Examinons les colonnes qui nous permettront de faire les jointures entre fichiers.


In [ ]:
print("Colonnes SIRH :")
print(df_sirh.columns.tolist())
print(
    f"\nClé potentielle 'id_employee' : {df_sirh['id_employee'].nunique()} valeurs uniques sur {len(df_sirh)} lignes"
)

print("\n" + "-" * 60)
print("\nColonnes Évaluations :")
print(df_eval.columns.tolist())
print(
    f"\nClé potentielle 'eval_number' : {df_eval['eval_number'].nunique()} valeurs uniques sur {len(df_eval)} lignes"
)

print("\n" + "-" * 60)
print("\nColonnes Sondage :")
print(df_sondage.columns.tolist())
print(
    f"\nClé potentielle 'code_sondage' : {df_sondage['code_sondage'].nunique()} valeurs uniques sur {len(df_sondage)} lignes"
)

In [ ]:
# Analysons le format des clés pour comprendre comment les relier
print("Exemples de clés :")
print(f"\nSIRH - id_employee (premiers) : {df_sirh['id_employee'].head(10).tolist()}")
print(f"\nEval - eval_number (premiers) : {df_eval['eval_number'].head(10).tolist()}")
print(
    f"\nSondage - code_sondage (premiers) : {df_sondage['code_sondage'].head(10).tolist()}"
)

#### Analyse du format des clés

Regardons de plus près comment sont structurées ces clés.


In [ ]:
print("Comparaison du nombre de lignes :")
print(f"  - SIRH : {len(df_sirh)} lignes")
print(f"  - Évaluations : {len(df_eval)} lignes")
print(f"  - Sondage : {len(df_sondage)} lignes")

# Si tous les fichiers ont le même nombre de lignes,
# ils correspondent probablement aux mêmes employés

#### Vérification de la cohérence des données

Comparons le nombre de lignes pour détecter d'éventuels problèmes.


In [ ]:
print("Recherche de colonnes communes entre les fichiers :\n")

sirh_cols = set(df_sirh.columns)
eval_cols = set(df_eval.columns)
sondage_cols = set(df_sondage.columns)

print(f"SIRH ∩ Évaluations : {sirh_cols.intersection(eval_cols)}")
print(f"SIRH ∩ Sondage : {sirh_cols.intersection(sondage_cols)}")
print(f"Évaluations ∩ Sondage : {eval_cols.intersection(sondage_cols)}")

print("\n" + "=" * 60)
if not sirh_cols.intersection(eval_cols) and not sirh_cols.intersection(sondage_cols):
    print("Résultat : Aucune colonne commune détectée")
    print("Les 3 fichiers ont des colonnes strictement différentes")

#### Comparaison visuelle des clés

Analysons la structure des clés pour identifier leur correspondance.


In [ ]:
print("Comparaison visuelle des premières valeurs de chaque clé :\n")

comparison_df = pd.DataFrame(
    {
        "id_employee": df_sirh["id_employee"].head(10),
        "eval_number": df_eval["eval_number"].head(10),
        "code_sondage": df_sondage["code_sondage"].head(10),
    }
)
print(comparison_df)

print("\n" + "=" * 60)
print("Observation : Les 3 clés suivent un pattern cohérent")
print("  - id_employee : valeurs numériques (1, 2, 3...)")
print("  - eval_number : format 'E_X' où X correspond à id_employee")
print("  - code_sondage : même valeur que id_employee")
print("\nConclusion : Les lignes sont alignées par leur position (index)")

#### Conclusion : Stratégie de fusion

**Constat :**

- Aucune colonne commune entre les 3 fichiers
- Même nombre de lignes (1470) dans chaque fichier
- Les clés suivent un pattern cohérent suggérant un alignement par index
- Chaque fichier contient des informations complémentaires :
  - SIRH → infos administratives (ancienneté, salaire, département...)
  - Évaluations → métriques de performance (notes, satisfaction...)
  - Sondage → perception des employés (stress, équilibre vie pro/perso...)

**Stratégie retenue :**

Concaténation horizontale par index avec `pd.concat([df_sirh, df_eval, df_sondage], axis=1)`

**Justification :**

- Les lignes sont déjà alignées (id_employee=1 ↔ eval_number="E_1" ↔ code_sondage=1)
- Pas besoin de jointure SQL complexe
- Les colonnes identifiantes seront conservées pour traçabilité


---

## 5. Fusion des données

Création du DataFrame central en fusionnant les 3 sources par concaténation horizontale.


#### Création du DataFrame central

Fusion des 3 fichiers par concaténation horizontale (axis=1).


In [ ]:
df_merged = pd.concat([df_sirh, df_eval, df_sondage], axis=1)

print("DataFrame fusionné créé :")
print(f"  - {df_merged.shape[0]} lignes")
print(f"  - {df_merged.shape[1]} colonnes")
print(f"\nColonnes : {df_merged.columns.tolist()}")

#### Gestion des colonnes dupliquées

Vérification et suppression des éventuelles colonnes en double.


In [ ]:
# Vérification des colonnes dupliquées
duplicated_cols = df_merged.columns[df_merged.columns.duplicated()].tolist()

if duplicated_cols:
    print(f"{len(duplicated_cols)} colonne(s) dupliquée(s) détectée(s) :")
    for col in duplicated_cols:
        print(f"  - {col}")

    # Suppression des doublons (on garde la première occurrence)
    df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]
    print("\nColonnes dupliquées supprimées")
    print(f"Nouvelles dimensions : {df_merged.shape}")
else:
    print("Aucune colonne dupliquée détectée")

#### Aperçu du DataFrame central

Visualisation des premières lignes du dataset fusionné.


In [ ]:
df_merged.head()

#### Structure du DataFrame fusionné

Informations sur les types de données et la mémoire.


In [ ]:
df_merged.info()

#### Analyse de la variable cible

Distribution de `a_quitte_l_entreprise` - la variable à prédire.


In [ ]:
print("Variable cible - 'a_quitte_l_entreprise' :")
print(f"Type : {df_merged['a_quitte_l_entreprise'].dtype}")
print("\nDistribution :")
print(df_merged["a_quitte_l_entreprise"].value_counts())
print("\nProportions (%) :")
print((df_merged["a_quitte_l_entreprise"].value_counts(normalize=True) * 100).round(2))

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Comptage
df_merged["a_quitte_l_entreprise"].value_counts().plot(
    kind="bar", ax=ax[0], color=["#2ecc71", "#e74c3c"]
)
ax[0].set_title("Distribution de la variable cible", fontsize=12, fontweight="bold")
ax[0].set_xlabel("A quitté l'entreprise")
ax[0].set_ylabel("Nombre d'employés")
ax[0].set_xticklabels(["Non", "Oui"], rotation=0)

# Proportions
df_merged["a_quitte_l_entreprise"].value_counts().plot(
    kind="pie",
    ax=ax[1],
    autopct="%1.1f%%",
    colors=["#2ecc71", "#e74c3c"],
    labels=["Restés", "Partis"],
)
ax[1].set_title("Proportions", fontsize=12, fontweight="bold")
ax[1].set_ylabel("")

plt.tight_layout()
plt.show()

print("OBSERVATION CRITIQUE : Déséquilibre des classes !")
print("     → À gérer en modélisation (stratification, class_weights, SMOTE)")


#### Synthèse : Variable cible

- **84% restés** vs **16% partis** → Ratio 5:1
- Déséquilibre à gérer : stratification, class_weight, resampling
- Accuracy insuffisante comme métrique (84% sans rien faire)


---

## 6. Vue d'ensemble du dataset central

Avant de comparer les employés partis vs restés, vérifions la qualité et la structure des données :

- Valeurs manquantes
- Types de colonnes (numériques vs catégorielles)
- Colonnes identifiantes à exclure de l'analyse


#### Analyse des valeurs manquantes

Vérifions s'il y a des données manquantes dans le dataset fusionné.


In [ ]:
# Analyse des valeurs manquantes
print("Analyse des valeurs manquantes :\n")

missing_values = df_merged.isnull().sum()
missing_pct = (df_merged.isnull().sum() / len(df_merged) * 100).round(2)

missing_df = pd.DataFrame(
    {"Valeurs manquantes": missing_values, "Pourcentage (%)": missing_pct}
)

# Afficher seulement les colonnes avec des valeurs manquantes
missing_with_values = missing_df[missing_df["Valeurs manquantes"] > 0]

if len(missing_with_values) > 0:
    print(f"{len(missing_with_values)} colonne(s) avec des valeurs manquantes :")
    print(missing_with_values.sort_values("Pourcentage (%)", ascending=False))
else:
    print("Aucune valeur manquante dans le dataset !")
    print(f"   → {df_merged.shape[0]} lignes × {df_merged.shape[1]} colonnes complètes")

#### Classification des colonnes par type

Identifions les colonnes numériques et catégorielles pour orienter l'analyse exploratoire.


In [ ]:
print("Classification des colonnes par type :\n")

# Colonnes identifiantes (à exclure de l'analyse)
id_cols = ["id_employee", "eval_number", "code_sondage"]

# Variable cible
target_col = "a_quitte_l_entreprise"

# Colonnes numériques (excluant les IDs)
numeric_cols = df_merged.select_dtypes(include=["int64", "float64"]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in id_cols + [target_col]]

# Colonnes catégorielles
categorical_cols = df_merged.select_dtypes(include=["object"]).columns.tolist()
categorical_cols = [
    col for col in categorical_cols if col not in id_cols + [target_col]
]

print(f"Colonnes identifiantes ({len(id_cols)}) - À EXCLURE :")
print(f"   {id_cols}\n")

print("Variable cible :")
print(f"   {target_col}\n")

print(f"Colonnes numériques ({len(numeric_cols)}) :")
print(f"   {numeric_cols}\n")

print(f"Colonnes catégorielles ({len(categorical_cols)}) :")
print(f"   {categorical_cols}")

print(f"Total features analysables : {len(numeric_cols) + len(categorical_cols)}")

#### Résumé structuré du dataset

Tableau récapitulatif avec le type, les valeurs uniques et des exemples pour chaque colonne.


In [ ]:
summary_data = []

for col in df_merged.columns:
    if col in id_cols:
        category = "Identifiant"
    elif col == target_col:
        category = "Cible"
    elif col in numeric_cols:
        category = "Numérique"
    else:
        category = "Catégorielle"

    summary_data.append(
        {
            "Colonne": col,
            "Catégorie": category,
            "Type": str(df_merged[col].dtype),
            "Valeurs uniques": df_merged[col].nunique(),
            "Exemple": str(df_merged[col].iloc[0])[:30],
        }
    )

summary_df = pd.DataFrame(summary_data)
summary_df

---

## 7. Analyse exploratoire comparative : Partis vs Restés

Objectif principal de cette section : **identifier les différences clés** entre les employés ayant quitté l'entreprise et ceux qui y sont restés.

Nous utiliserons **Plotly** pour des graphiques interactifs.


#### Import de Plotly et préparation des données

Configuration de Plotly et création d'une colonne lisible pour la variable cible.


In [ ]:
df_merged["statut"] = df_merged["a_quitte_l_entreprise"].map(
    {"Oui": "Parti", "Non": "Resté"}
)

colors = {"Resté": "#2ecc71", "Parti": "#e74c3c"}

print(f"   Distribution : {df_merged['statut'].value_counts().to_dict()}")

### 7.1 Analyse des variables numériques

Comparons les **moyennes** des variables numériques entre les employés partis et restés avec un graphique unique et lisible.


#### Observations : Variables numériques

**Principales différences observées :**

| Variable                      | Différence | Observation                          |
| ----------------------------- | ---------- | ------------------------------------ |
| `nombre_participation_pee`    | -37.6%     | Participation PEE plus faible        |
| `annees_dans_le_poste_actuel` | -35.3%     | Ancienneté dans le poste plus faible |
| `revenu_mensuel`              | -29.9%     | Salaire plus bas                     |
| `distance_domicile_travail`   | +19.3%     | Distance plus grande                 |

**Note :** Ce sont des observations descriptives. Le modèle confirmera l'importance réelle de chaque variable.


---

## Synthèse Partie 1 : Analyse Exploratoire

### Données fusionnées

| Métrique           | Valeur                       |
| ------------------ | ---------------------------- |
| Employés           | 1470                         |
| Variables          | 34 colonnes                  |
| Sources            | SIRH + Évaluations + Sondage |
| Valeurs manquantes | 0                            |

### Variable cible : Déséquilibre critique

| Classe | Effectif | Proportion |
| ------ | -------- | ---------- |
| Restés | 1233     | **84%**    |
| Partis | 237      | **16%**    |

→ **Ratio 5:1** : nécessite stratification + gestion du déséquilibre (class_weight, SMOTE)

### Profil type de l'employé qui part

**Variables numériques discriminantes :**

| Variable                      | Écart vs Restés | Interprétation RH            |
| ----------------------------- | --------------- | ---------------------------- |
| `nombre_participation_pee`    | **-37.6%**      | Moins engagés financièrement |
| `annees_dans_le_poste_actuel` | **-35.3%**      | Moins d'ancienneté poste     |
| `revenu_mensuel`              | **-29.9%**      | Salaire plus bas             |
| `distance_domicile_travail`   | **+19.3%**      | Trajet plus long             |

**Variables catégorielles à risque :**

| Variable                | Modalité à risque       | Taux de churn |
| ----------------------- | ----------------------- | ------------- |
| `poste`                 | Représentant Commercial | **39.8%**     |
| `heure_supplementaires` | Oui                     | **30.5%**     |
| `statut_marital`        | Célibataire             | **25.5%**     |
| `frequence_deplacement` | Fréquent                | Taux élevé    |

**Profils stables (faible churn) :**

- Directeur Technique (2.5%), Manager (6.9%)
- Pas d'heures sup (10.4%)
- Mariés, ancienneté élevée

### Insights métier pour les RH

1. **Rémunération** : Les employés qui partent gagnent ~30% de moins → Revoir la politique salariale
2. **Heures sup** : 30% de churn chez ceux qui en font → Surveiller la charge de travail
3. **Mobilité** : Distance domicile-travail corrélée au départ → Télétravail comme levier
4. **Engagement** : Faible participation PEE = signal d'alerte → Renforcer l'intéressement
5. **Postes à risque** : Commerciaux = 40% de turnover → Actions ciblées

### Variables retenues pour la modélisation

**Numériques potentiellement prédictives :**

- `revenu_mensuel`, `annees_dans_le_poste_actuel`, `nombre_participation_pee`
- `distance_domicile_travail`, `satisfaction_*` (4 variables)

**Catégorielles potentiellement prédictives :**

- `heure_supplementaires`, `poste`, `statut_marital`, `frequence_deplacement`

**⚠️ Attention** : Ces observations sont **descriptives**. Le modèle (puis SHAP) confirmera l'importance réelle de chaque variable.


---

# Partie 2 : Feature Engineering

Dans cette partie, nous allons :

1. **Nettoyer les donnees** : doublons, outliers, colonnes inutiles
2. **Analyser les correlations** : matrice de Pearson, suppression des variables trop correlees
3. **Encoder les variables categorielles** : OneHotEncoder pour les modeles
4. **Creer X et y** : preparation finale pour la modelisation


## 8. Nettoyage des donnees

### 8.1 Verification des doublons


In [ ]:
print(f"  Nombre de lignes dupliquees : {df_merged.duplicated().sum()}")
print(f"\nDoublons sur 'id_employee' : {df_merged['id_employee'].duplicated().sum()}")

### 8.2 Detection des outliers (methode IQR)

**Qu'est-ce que la methode IQR (Interquartile Range) ?**

L'IQR est une methode statistique robuste pour detecter les valeurs aberrantes :

**Calcul des bornes :**

- Borne inferieure = Q1 - 1.5 x IQR
- Borne superieure = Q3 + 1.5 x IQR

Toute valeur en dehors de ces bornes est consideree comme un **outlier**.

**Pourquoi IQR plutot que Z-score ?**

- IQR est base sur les **quartiles** (pas la moyenne)
- Donc **insensible aux valeurs extremes** elles-memes
- Plus adapte aux distributions non-normales


In [ ]:
# Detection des outliers avec la methode IQR (Interquartile Range)
def detect_outliers_iqr(df, columns):
    """Detecte les outliers pour chaque colonne numerique avec la methode IQR."""
    outliers_summary = []

    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        n_outliers = len(outliers)
        pct_outliers = (n_outliers / len(df)) * 100

        if n_outliers > 0:
            outliers_summary.append(
                {
                    "Variable": col,
                    "Nb outliers": n_outliers,
                    "% outliers": round(pct_outliers, 1),
                    "Borne inf": round(lower_bound, 2),
                    "Borne sup": round(upper_bound, 2),
                    "Min reel": round(df[col].min(), 2),
                    "Max reel": round(df[col].max(), 2),
                }
            )

    return pd.DataFrame(outliers_summary)


# Exclure les colonnes ID et la cible pour l'analyse des outliers
cols_to_check = [
    col
    for col in numeric_cols
    if col not in ["id_employee", "eval_number", "code_sondage"]
]
outliers_df = detect_outliers_iqr(df_merged, cols_to_check)

print(f"Variables avec outliers : {len(outliers_df)} / {len(cols_to_check)}")
print()
if len(outliers_df) > 0:
    display(outliers_df.sort_values("% outliers", ascending=False))

**Decision sur les outliers :**

Les outliers detectes sont des valeurs coherentes dans un contexte RH :

- **Revenus eleves** : salaires de cadres superieurs (jusqu'a 19 999 EUR)
- **Anciennete elevee** : employes fideles (jusqu'a 40 ans)
- **Formations** : 6 formations maximum, valeur plausible

Ces valeurs ne sont pas des erreurs de saisie mais des cas legitimes. Nous les **conservons** car :

1. Les modeles tree-based (Random Forest, XGBoost) gerent bien les outliers
2. Ces profils extremes peuvent etre pertinents pour predire le churn


### 8.3 Identification des colonnes a supprimer

**Pourquoi supprimer certaines colonnes ?**

1. **Colonnes ID** (id_employee, eval_number, code_sondage)
   - Ce sont des identifiants uniques (1, 2, 3...)
   - Aucune valeur predictive : le modele ne peut pas apprendre que "employe 42" part plus souvent

2. **Colonnes a variance nulle**
   - Une colonne avec la **meme valeur pour tous** (ex: `nombre_heures_travailless = 80` pour tout le monde)
   - Aucune information discriminante : impossible de differencier partis vs restes

3. **Colonnes redondantes**
   - `a_quitte_l_entreprise` et `statut` contiennent la meme information que notre cible
   - Les garder = **data leakage** (le modele "triche")


In [ ]:
print("Colonnes actuelles du dataset :")
print(df_merged.columns.tolist())

id_columns = ["id_employee", "eval_number", "code_sondage"]

target_column = "depart"

# Colonnes a variance nulle ou quasi-nulle
print("\n" + "-" * 60)
print("\nVerification des colonnes a faible variance :")
for col in df_merged.columns:
    unique_ratio = df_merged[col].nunique() / len(df_merged)
    if df_merged[col].nunique() <= 2 and col != target_column:
        print(
            f"  {col} : {df_merged[col].nunique()} valeurs uniques -> {df_merged[col].value_counts().to_dict()}"
        )

In [ ]:
columns_to_drop = {
    # Colonnes ID (pas de valeur predictive)
    "id_employee": "Identifiant unique - pas de valeur predictive",
    "eval_number": "Identifiant evaluation - pas de valeur predictive",
    "code_sondage": "Identifiant sondage - pas de valeur predictive",
    # Colonnes a variance nulle (meme valeur pour tous)
    "nombre_heures_travailless": "Variance nulle - toujours 80",
    "nombre_employee_sous_responsabilite": "Variance nulle - toujours 1",
    "ayant_enfants": "Variance nulle - toujours Y",
    # Colonne redondante avec la cible
    "a_quitte_l_entreprise": 'Redondante avec "statut" (variable cible)',
    "statut": 'Redondante - nous utiliserons "depart" comme cible binaire',
}

print("Colonnes a supprimer :")
for col, reason in columns_to_drop.items():
    print(f"  - {col}: {reason}")

print(f"\nTotal : {len(columns_to_drop)} colonnes a supprimer")

In [ ]:
# Creation du DataFrame nettoye
df_clean = df_merged.copy()

# Creation de la variable cible binaire 'depart' (0 = Reste, 1 = Churn)
df_clean["depart"] = (df_clean["statut"] == "Parti").astype(int)

# Suppression des colonnes identifiees
df_clean = df_clean.drop(columns=list(columns_to_drop.keys()))

print(f"Dataset initial : {df_merged.shape[0]} lignes x {df_merged.shape[1]} colonnes")
print(f"Dataset nettoye : {df_clean.shape[0]} lignes x {df_clean.shape[1]} colonnes")
print(f"\nColonnes restantes ({df_clean.shape[1]}) :")
print(df_clean.columns.tolist())

## 9. Analyse des correlations

**Pourquoi analyser les correlations ?**

1. **Identifier les relations lineaires** entre variables
2. **Detecter la multicolinearite** : si 2 variables sont tres correlees (|r| > 0.7), elles apportent la meme information → on peut en supprimer une
3. **Comprendre les liens avec la cible** : quelles variables sont les plus correlees avec le depart ?

### 9.1 Matrice de correlation de Pearson

**Pearson** mesure les correlations **lineaires** :

- r = +1 : correlation positive parfaite
- r = 0 : pas de correlation lineaire
- r = -1 : correlation negative parfaite


In [ ]:
# Calcul de la matrice de correlation Pearson
numeric_cols_clean = df_clean.select_dtypes(
    include=["int64", "float64"]
).columns.tolist()
print(f"Variables numériques pour la corrélation : {len(numeric_cols_clean)}")

corr_matrix = df_clean[numeric_cols_clean].corr()

# Visualisation avec matplotlib/seaborn
fig, ax = plt.subplots(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    fmt=".2f",
    cmap="RdBu_r",
    center=0,
    square=True,
    linewidths=0.5,
    ax=ax,
    annot_kws={"size": 7},
    cbar_kws={"shrink": 0.8},
)
plt.title("Matrice de Corrélation de Pearson", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

# Corrélations avec la cible
print("\nCORRÉLATIONS AVEC LA CIBLE 'depart' :")
print("-" * 50)
target_corr = corr_matrix["depart"].drop("depart").sort_values(key=abs, ascending=False)
for var, corr_value in target_corr.items():
    print(f"   {var:40} : {corr_value:+.3f}")

### 9.2 Identification des correlations fortes (|r| > 0.7)

**Pourquoi analyser les corrélations ?**

Si deux variables sont **très corrélées** (ex: `revenu_mensuel` et `niveau_hierarchique_poste` à 0.95), elles apportent une information **partiellement redondante**.

**Faut-il supprimer ces variables ?**

⚠️ **Non !** Contrairement à une idée reçue, la **multicolinéarité n'est pas un problème** pour les modèles basés sur les arbres (Random Forest, LightGBM) que nous utilisons :

- Ces modèles **n'utilisent pas de coefficients linéaires** → pas d'instabilité
- Même avec 95% de corrélation, il reste **5% d'information unique** qui peut être utile
- Supprimer une variable = **perte potentielle d'information prédictive**
- **Pas de data leakage** : ces variables sont des caractéristiques RH légitimes

**Conclusion :** On **conserve toutes les variables** et on laisse le modèle choisir les plus pertinentes.


In [ ]:
# Identification des paires de variables fortement correlees (|r| > 0.7)
threshold = 0.7
high_corr_pairs = []

# Parcourir le triangle inferieur de la matrice (sans la diagonale)
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            high_corr_pairs.append(
                {
                    "Variable 1": corr_matrix.columns[j],
                    "Variable 2": corr_matrix.columns[i],
                    "Correlation": round(corr_matrix.iloc[i, j], 3),
                }
            )

high_corr_df = pd.DataFrame(high_corr_pairs).sort_values("Correlation", ascending=False)
print(f"Paires de variables avec correlation |r| > {threshold} :")
print()
display(high_corr_df)

In [ ]:
# Analyse des corrélations avec la cible 'depart'
# (pour information - toutes les variables sont conservées)

print("Corrélation avec la cible 'depart' :")
target_corr = corr_matrix["depart"].drop("depart").abs().sort_values(ascending=False)
print(target_corr.head(10))

print("\n" + "-" * 60)
print("\nAnalyse des paires fortement corrélées :")
for _, row in high_corr_df.iterrows():
    var1, var2 = row["Variable 1"], row["Variable 2"]
    corr1 = abs(corr_matrix.loc["depart", var1])
    corr2 = abs(corr_matrix.loc["depart", var2])
    print(
        f"\n{var1} (|r| avec depart = {corr1:.3f}) vs {var2} (|r| avec depart = {corr2:.3f})"
    )

In [ ]:
# ⚠️ DECISION : On garde TOUTES les variables corrélées
# Justification métier :
# - Même avec r=0.95, il reste 5% d'information unique potentiellement utile
# - Les modèles à base d'arbres (RF, LightGBM) gèrent bien la multicolinéarité
# - Pas de data leakage → pas de raison de supprimer

print("✅ DÉCISION : Conservation de TOUTES les variables")
print()
print("Variables fortement corrélées (|r| > 0.7) :")
for _, row in high_corr_df.iterrows():
    print(
        f"   {row['Variable 1']} ↔ {row['Variable 2']} : r = {row['Correlation']:.2f}"
    )
print()
print("Justification :")
print("   - Les modèles à base d'arbres ne sont pas affectés par la multicolinéarité")
print("   - Même 5% d'information unique peut améliorer la prédiction")
print("   - Aucun data leakage détecté → variables RH légitimes")
print()
print(f"Dataset conservé : {df_clean.shape[0]} lignes x {df_clean.shape[1]} colonnes")

### 9.3 Pourquoi conserver les variables corrélées ?

**Analyse des corrélations détectées (|r| > 0.7) :**

| Variable 1                 | Variable 2                      | Corrélation |
| -------------------------- | ------------------------------- | ----------- |
| `revenu_mensuel`           | `niveau_hierarchique_poste`     | **0.95**    |
| `annee_experience_totale`  | `niveau_hierarchique_poste`     | 0.78        |
| `annees_dans_l_entreprise` | `annees_dans_le_poste_actuel`   | 0.76        |
| `annees_dans_l_entreprise` | `annes_sous_responsable_actuel` | 0.77        |

**Pourquoi NE PAS supprimer ces variables ?**

1. **Modèles à base d'arbres** (RF, LightGBM) :
   - Ne calculent pas de coefficients linéaires → pas d'instabilité
   - Sélectionnent automatiquement les features les plus discriminantes
   - La corrélation ne pose pas de problème d'estimation

2. **Information résiduelle** :
   - Même avec r=0.95, il reste **5% d'information unique**
   - Cette information peut être utile pour certains profils d'employés
   - Supprimer = risque de perdre du pouvoir prédictif

3. **Interprétabilité SHAP** :
   - SHAP répartit équitablement l'importance entre variables corrélées
   - On peut voir les deux perspectives : salaire ET niveau hiérarchique

4. **Pas de data leakage** :
   - Ces variables sont des caractéristiques RH disponibles avant le départ
   - Aucune fuite d'information future


### 9.4 Conversion de `augementation_salaire_precedente` en numérique

**Problème identifié :** La colonne `augementation_salaire_precedente` contient des valeurs comme "11 %", "23 %" qui sont stockées comme **texte (object)**.

**Pourquoi convertir en numérique ?**

| Approche                      | Nb features          | Compréhension modèle             | Risque overfitting |
| ----------------------------- | -------------------- | -------------------------------- | ------------------ |
| **Catégorielle (défaut)**     | 14 colonnes (OneHot) | ❌ Perd l'ordre et les distances | ⚠️ Plus élevé      |
| **Numérique (best practice)** | 1 colonne            | ✅ 11% < 12% < 23%               | ✅ Réduit          |

**Avantages de la conversion :**

- Le modèle comprend que **23% > 11%** (relation ordinale préservée)
- Le modèle comprend que **23% - 11% = 12 points** (distances préservées)
- **1 feature** au lieu de **14** → moins de dimensions → moins d'overfitting
- Coefficient plus **interprétable** : "+1% d'augmentation = X% de risque de départ"


In [ ]:
# Conversion de "11 %" -> 11.0 (valeur numerique)
# On garde la valeur en pourcentage (11, 12, 23...) plutot qu'en decimal (0.11, 0.12...)

print("Avant conversion :")
print(f"  Type: {df_clean['augementation_salaire_precedente'].dtype}")
print(
    f"  Valeurs uniques: {df_clean['augementation_salaire_precedente'].unique()[:5]}..."
)

# Suppression du " %" et conversion en float
df_clean["augementation_salaire_precedente"] = (
    df_clean["augementation_salaire_precedente"]
    .str.replace(" %", "", regex=False)
    .astype(float)
)

print("\nApres conversion :")
print(f"  Type: {df_clean['augementation_salaire_precedente'].dtype}")
print(
    f"  Valeurs uniques: {sorted(df_clean['augementation_salaire_precedente'].unique())}"
)
print(f"  Min: {df_clean['augementation_salaire_precedente'].min()}%")
print(f"  Max: {df_clean['augementation_salaire_precedente'].max()}%")
print(f"  Moyenne: {df_clean['augementation_salaire_precedente'].mean():.1f}%")

print("\nLa colonne sera maintenant traitee comme NUMERIQUE dans le Pipeline")
print("   → StandardScaler au lieu de OneHotEncoder")
print("   → 1 feature au lieu de 14")

## 10. Feature Engineering - Creation de nouvelles variables

**Pourquoi creer de nouvelles features ?**

"features supplementaires par rapport aux donnees d'origine"

**Objectif :** Creer des variables qui capturent des **informations metier** que les colonnes brutes ne montrent pas directement.

**Features creees (3) :**

| Feature                    | Formule                                        | Interpretation metier                            |
| -------------------------- | ---------------------------------------------- | ------------------------------------------------ |
| `ratio_salaire_experience` | revenu_mensuel / (experience + 1)              | Employe sous-paye par rapport a son experience ? |
| `stagnation_poste`         | annees_dans_le_poste - annees_depuis_promotion | Employe bloque sans evolution ?                  |
| `satisfaction_globale`     | moyenne des 4 satisfactions employee           | Score synthetique de bien-etre                   |

**Pourquoi seulement 3 ?**

- Eviter l'**overfitting** (trop de features pour peu de donnees)
- Chaque feature doit avoir un **sens metier RH**
- Le dataset est deja riche (50+ colonnes apres encodage)


In [ ]:
# Feature 1 : Ratio salaire / experience
# Interpretation : Un ratio bas = potentiellement sous-paye
# Note: +1 pour eviter division par zero si experience = 0
df_clean["ratio_salaire_experience"] = df_clean["revenu_mensuel"] / (
    df_clean["annee_experience_totale"] + 1
)

# Feature 2 : Stagnation de carriere
# Interpretation : Valeur elevee = beaucoup d'annees dans le poste sans promotion recente
df_clean["stagnation_poste"] = (
    df_clean["annees_dans_le_poste_actuel"]
    - df_clean["annees_depuis_la_derniere_promotion"]
)

# Feature 3 : Satisfaction globale (moyenne des 4 satisfactions)
# Interpretation : Score synthetique qui resume le bien-etre au travail
cols_satisfaction = [
    "satisfaction_employee_environnement",
    "satisfaction_employee_nature_travail",
    "satisfaction_employee_equipe",
    "satisfaction_employee_equilibre_pro_perso",
]
df_clean["satisfaction_globale"] = df_clean[cols_satisfaction].mean(axis=1)

print("3 features creees avec succes !")
print(f"\nNouvelle shape du dataframe : {df_clean.shape}")
print("\nApercu des nouvelles features :")
df_clean[
    ["ratio_salaire_experience", "stagnation_poste", "satisfaction_globale"]
].describe()

### 10.2 Verification de la pertinence des features

Verifions la correlation de nos nouvelles features avec la variable cible `depart` :


In [ ]:
# Correlation des nouvelles features avec depart (variable cible)
new_features = ["ratio_salaire_experience", "stagnation_poste", "satisfaction_globale"]

# Creer un dataframe temporaire avec les nouvelles features et la cible
temp_df = df_clean[new_features].copy()
temp_df["depart"] = df_clean[
    "depart"
].values  # La cible est encore dans df_clean a ce stade

correlations = temp_df.corr()["depart"].drop("depart")

print("Correlation avec depart (variable cible) :")
for feat, corr in correlations.items():
    signe = "🔴" if corr > 0 else "🟢"
    interpretation = "quitte plus" if corr > 0 else "reste plus"
    print(f"{signe} {feat}: {corr:.4f} ({interpretation})")

print("\nInterpretation :")
print("   - satisfaction_globale : plus les gens sont satisfaits, moins ils partent")
print(
    "   - stagnation_poste : correlation negative = ceux qui stagnent RESTENT (profils seniors stables)"
)
print(
    "   - ratio_salaire_experience : les mieux payes par rapport a leur experience partent plus"
)

## 11. Pipeline de preprocessing avec ColumnTransformer

### Pourquoi utiliser un Pipeline ?

| Avantage                     | Explication                                                               |
| ---------------------------- | ------------------------------------------------------------------------- |
| ✅ **Evite le data leakage** | Le preprocessing est applique UNIQUEMENT sur le train a chaque fold de CV |
| ✅ **Code propre**           | Tout le preprocessing est encapsule dans un seul objet                    |
| ✅ **Reproductible**         | Facile a reutiliser et deployer                                           |
| ✅ **Compatible CV**         | S'integre parfaitement avec `cross_val_score` et `GridSearchCV`           |

### 11.1 Identification des colonnes


In [ ]:
# Separation features / cible AVANT le pipeline
y = df_clean["depart"]
X = df_clean.drop(columns=["depart"])

# Identification automatique des colonnes numeriques et categorielles
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

print("Colonnes identifiees pour le Pipeline :")
print(
    f"\n  Numeriques ({len(num_cols)}) : {num_cols[:5]}{'...' if len(num_cols) > 5 else ''}"
)
print(f"\n  Categorielles ({len(cat_cols)}) :")
for col in cat_cols:
    unique_vals = X[col].unique()
    print(f"    - {col} ({len(unique_vals)} modalites)")

### 11.2 Creation du ColumnTransformer

**ColumnTransformer** applique des transformations differentes selon le type de colonne :

| Type de colonne  | Transformation                | Parametre                  |
| ---------------- | ----------------------------- | -------------------------- |
| **Numerique**    | `StandardScaler()`            | Moyenne=0, Ecart-type=1    |
| **Categorielle** | `OneHotEncoder(drop='first')` | Evite colinearite parfaite |

**Parametre `remainder='passthrough'`** : conserve les colonnes non transformees (si elles existent).


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Creation du ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),  # Standardisation des numeriques
        (
            "cat",
            OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False),
            cat_cols,
        ),  # Encodage des categorielles
    ],
    remainder="passthrough",  # Conserver les autres colonnes si elles existent
)

print("\nTransformations definies :")
print(f"  - 'num' : StandardScaler sur {len(num_cols)} colonnes numeriques")
print(f"  - 'cat' : OneHotEncoder sur {len(cat_cols)} colonnes categorielles")
print("\nLe preprocessor sera FIT sur X_train uniquement (pas de data leakage)")

### 11.3 Split Train/Test avec stratification

**❓ Question légitime : Pourquoi faire un split si on fait de la cross-validation ?**

On fait les **DEUX** pour des raisons différentes :

| Étape                                       | Objectif                                              | Utilisation                          |
| ------------------------------------------- | ----------------------------------------------------- | ------------------------------------ |
| **Split Train/Test (80/20)**                | Avoir un **jeu de test FINAL jamais touché**          | Évaluation finale du meilleur modèle |
| **Cross-validation (sur Train uniquement)** | **Comparer les modèles** et tuner les hyperparamètres | Sélection du meilleur modèle         |

**Pourquoi ?** Si on fait la CV sur **tout le dataset**, on n'a plus de données "fraîches" pour vérifier si le modèle généralise vraiment. Le test est le **juge final impartial**.

**Workflow preprocessing (éviter data leakage) :**

| Etape | Action                                   | Explication                                    |
| ----- | ---------------------------------------- | ---------------------------------------------- |
| 1     | `X, y = separation features/cible`       | Séparer les variables explicatives de la cible |
| 2     | `X_train, X_test = split(X, y)`          | Split **avant** preprocessing                  |
| 3     | `preprocessor.fit(X_train)`              | Fit sur train **UNIQUEMENT**                   |
| 4     | `X_train_processed = transform(X_train)` | Transform train                                |
| 5     | `X_test_processed = transform(X_test)`   | Transform test (mêmes paramètres du train)     |


In [ ]:
from sklearn.model_selection import train_test_split

# Split stratifie AVANT le fit du preprocessor
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,  # Important pour le desequilibre de classes
)

print("Split train/test avec stratification :")
print(f"\n  Train : {X_train.shape[0]} lignes ({X_train.shape[0] / len(X) * 100:.0f}%)")
print(f"    - Churn : {y_train.sum()} ({y_train.mean() * 100:.1f}%)")
print(
    f"    - Non-churn : {len(y_train) - y_train.sum()} ({(1 - y_train.mean()) * 100:.1f}%)"
)

print(f"\n  Test : {X_test.shape[0]} lignes ({X_test.shape[0] / len(X) * 100:.0f}%)")
print(f"    - Churn : {y_test.sum()} ({y_test.mean() * 100:.1f}%)")
print(
    f"    - Non-churn : {len(y_test) - y_test.sum()} ({(1 - y_test.mean()) * 100:.1f}%)"
)

### 11.4 Application du preprocessor (fit_transform sur train, transform sur test)

**C'est ici que la magie du Pipeline opere :**

- `fit_transform(X_train)` : calcule les parametres (moyenne, ecart-type, modalites) ET transforme
- `transform(X_test)` : utilise les parametres du train pour transformer le test

**Avantage majeur** : Quand on utilisera `cross_val_score`, le fit sera automatiquement refait sur chaque fold !


In [ ]:
# Fit sur train, transform sur train et test
X_train_processed = preprocessor.fit_transform(X_train)  # FIT + TRANSFORM
X_test_processed = preprocessor.transform(X_test)  # TRANSFORM seulement

# Recuperation des noms de colonnes pour lisibilite
# (Les colonnes numeriques gardent leur nom, les categorielles sont encodees)
num_feature_names = num_cols
cat_feature_names = (
    preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols).tolist()
)
all_feature_names = num_feature_names + cat_feature_names

print("Preprocessing applique avec succes !")
print("\nDimensions apres transformation :")
print(f"  X_train_processed : {X_train_processed.shape}")
print(f"  X_test_processed  : {X_test_processed.shape}")

print(f"\nNombre de features finales : {len(all_feature_names)}")
print(f"  - Numeriques (standardisees) : {len(num_feature_names)}")
print(f"  - Categorielles (encodees)   : {len(cat_feature_names)}")

### 11.5 Verification : pas de data leakage

Verifions que le StandardScaler a bien ete fit sur le train uniquement :

- **Train** : moyenne ≈ 0, ecart-type ≈ 1
- **Test** : moyenne ≠ 0 exactement (normal, car les parametres viennent du train)


In [ ]:
num_indices = list(range(len(num_cols)))

print("Verification du StandardScaler (colonnes numeriques) :")
print(f"\n  Train - Moyenne  : {X_train_processed[:, num_indices].mean():.6f}")
print(f"  Train - Std      : {X_train_processed[:, num_indices].std():.6f}")
print(f"\n  Test  - Moyenne  : {X_test_processed[:, num_indices].mean():.6f}")
print(f"  Test  - Std      : {X_test_processed[:, num_indices].std():.6f}")

print("\nPas de data leakage : le test n'a pas exactement moyenne=0")
print("   (les parametres du scaler viennent du train)")

---

## Synthèse Partie 2 : Feature Engineering

### Nettoyage effectué

- ✅ Aucun doublon détecté
- ✅ Outliers conservés (valeurs RH légitimes : hauts salaires, ancienneté élevée)
- ✅ 8 colonnes supprimées : identifiants, variance nulle, redondantes avec la cible

### Analyse des corrélations

- Matrice de **Pearson** (corrélations linéaires) - visualisation Plotly
- Matrice de **Spearman** (corrélations monotones)
- Corrélations fortes détectées (|r| > 0.7) mais **variables conservées**
- **Justification** : les modèles à base d'arbres gèrent bien la multicolinéarité, et même 5% d'information unique peut être utile

### Conversion de type (best practice)

- ✅ `augementation_salaire_precedente` : "11 %" → 11.0 (numérique)
- **Avantage** : 1 feature standardisée au lieu de 14 colonnes OneHot → moins d'overfitting
- Le modèle comprend que 23% > 11% (ordre préservé)

### Feature Engineering (3 nouvelles features métier)

| Feature                    | Formule                 | Corr. avec depart | Interprétation                      |
| -------------------------- | ----------------------- | ----------------- | ----------------------------------- |
| `ratio_salaire_experience` | salaire / (exp + 1)     | +0.10             | Bien payés partent plus             |
| `stagnation_poste`         | ancienneté - promotion  | -0.15             | Stagnants restent (profils stables) |
| `satisfaction_globale`     | moyenne 4 satisfactions | -0.16             | Satisfaits restent                  |

### Pipeline de preprocessing (ColumnTransformer)

| Type       | Transformation                | Colonnes             |
| ---------- | ----------------------------- | -------------------- |
| Numérique  | `StandardScaler()`            | 23 colonnes          |
| Catégoriel | `OneHotEncoder(drop='first')` | 7 cols → 21 features |

### Dataset prêt pour la modélisation

| Métrique         | Valeur                             |
| ---------------- | ---------------------------------- |
| Features totales | **44** (23 num + 21 cat)           |
| Train            | 1176 lignes (80%)                  |
| Test             | 294 lignes (20%)                   |
| Churn train      | 16.2%                              |
| Churn test       | 16.0%                              |
| Data leakage     | Vérifié (fit sur train uniquement) |


# Partie 3 : Modélisation de Référence (Baseline)

**Objectif :** Établir des **modèles de référence** pour comprendre la difficulté du problème avant d'optimiser.

---

## Approche méthodologique

| Étape | Modèle                     | Objectif                                                  |
| ----- | -------------------------- | --------------------------------------------------------- |
| 1     | **DummyClassifier**        | Baseline naïf (que vaut "toujours prédire majoritaire" ?) |
| 2     | **LogisticRegression**     | Modèle linéaire simple                                    |
| 3     | **RandomForestClassifier** | Modèle non-linéaire (arbres)                              |

## Métriques utilisées

Pour un problème de **classification binaire déséquilibrée** (16% churn), l'**accuracy** est trompeuse.

| Métrique      | Formule               | Interprétation métier                      |
| ------------- | --------------------- | ------------------------------------------ |
| **Precision** | TP / (TP + FP)        | "Parmi les alertes, combien sont vraies ?" |
| **Recall**    | TP / (TP + FN)        | "Combien de départs réels détectés ?"      |
| **F1-Score**  | 2 × (P × R) / (P + R) | Équilibre Precision/Recall                 |

**Contexte RH :** Le **Recall** est critique car **rater un départ** (FN) coûte plus cher qu'une fausse alerte (FP).


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import (
    make_scorer,
    recall_score,
    precision_score,
    f1_score,
    roc_auc_score,
)
from sklearn.base import clone


def evaluate_model_cv(
    model,
    X_data,
    y_data,
    preprocessor,
    resampler=None,
    cv=5,
    model_name="Model",
    pos_label=1,
):
    """
    Évalue un modèle avec validation croisée stratifiée.
    Si resampler est fourni (SMOTE, RandomUnderSampler...), l'applique sur le train.

    Parameters:
    -----------
    model : estimator sklearn
    X_data : DataFrame des features
    y_data : Series de la cible
    preprocessor : ColumnTransformer pour le preprocessing
    resampler : imblearn resampler (SMOTE, RandomUnderSampler...) ou None
    cv : int, nombre de folds
    model_name : str, nom du modèle pour l'affichage
    pos_label : int, label de la classe positive (défaut: 1)

    Returns:
    --------
    dict : résultats avec moyennes et écarts-types (test uniquement si resampling)
    """
    from sklearn.pipeline import Pipeline

    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    # Si pas de resampling, utiliser cross_validate classique (plus rapide)
    if resampler is None:
        pipeline = Pipeline([("preprocessor", preprocessor), ("classifier", model)])

        scoring = {
            "recall": make_scorer(recall_score, pos_label=pos_label),
            "precision": make_scorer(
                precision_score, pos_label=pos_label, zero_division=0
            ),
            "f1": make_scorer(f1_score, pos_label=pos_label),
            "roc_auc": "roc_auc",
        }

        cv_results = cross_validate(
            pipeline,
            X_data,
            y_data,
            cv=skf,
            scoring=scoring,
            return_train_score=True,
            n_jobs=-1,
        )

        results = {
            "model": model_name,
            "recall_train_mean": cv_results["train_recall"].mean(),
            "recall_train_std": cv_results["train_recall"].std(),
            "recall_test_mean": cv_results["test_recall"].mean(),
            "recall_test_std": cv_results["test_recall"].std(),
            "precision_train_mean": cv_results["train_precision"].mean(),
            "precision_train_std": cv_results["train_precision"].std(),
            "precision_test_mean": cv_results["test_precision"].mean(),
            "precision_test_std": cv_results["test_precision"].std(),
            "f1_train_mean": cv_results["train_f1"].mean(),
            "f1_train_std": cv_results["train_f1"].std(),
            "f1_test_mean": cv_results["test_f1"].mean(),
            "f1_test_std": cv_results["test_f1"].std(),
            "roc_auc_train_mean": cv_results["train_roc_auc"].mean(),
            "roc_auc_test_mean": cv_results["test_roc_auc"].mean(),
        }

        # Affichage avec train et test
        print(f"\n{'=' * 60}")
        print(f" {model_name}")
        print(f"{'=' * 60}")
        print(f"\n{'Métrique':<15} {'Train':>20} {'Test':>20}")
        print("-" * 55)
        print(
            f"{'Recall':<15} {results['recall_train_mean']:.3f} ± {results['recall_train_std']:.3f}    {results['recall_test_mean']:.3f} ± {results['recall_test_std']:.3f}"
        )
        print(
            f"{'Precision':<15} {results['precision_train_mean']:.3f} ± {results['precision_train_std']:.3f}    {results['precision_test_mean']:.3f} ± {results['precision_test_std']:.3f}"
        )
        print(
            f"{'F1-Score':<15} {results['f1_train_mean']:.3f} ± {results['f1_train_std']:.3f}    {results['f1_test_mean']:.3f} ± {results['f1_test_std']:.3f}"
        )
        print(
            f"{'ROC-AUC':<15} {results['roc_auc_train_mean']:.3f}              {results['roc_auc_test_mean']:.3f}"
        )
    else:
        # Avec resampling : boucle manuelle, métriques TEST uniquement
        # (prédire sur train n'a pas de sens car le modèle a déjà vu ces données)
        y_array = y_data.values if hasattr(y_data, "values") else y_data

        metrics = {
            "recall_test": [],
            "precision_test": [],
            "f1_test": [],
            "roc_auc_test": [],
        }

        for train_idx, test_idx in skf.split(X_data, y_array):
            X_train_fold, X_test_fold = X_data.iloc[train_idx], X_data.iloc[test_idx]
            y_train_fold, y_test_fold = y_array[train_idx], y_array[test_idx]

            # Preprocessing
            X_train_processed = preprocessor.fit_transform(X_train_fold)
            X_test_processed = preprocessor.transform(X_test_fold)

            # Resampling sur train uniquement
            X_train_resampled, y_train_resampled = resampler.fit_resample(
                X_train_processed, y_train_fold
            )

            # Entraînement
            clf = clone(model)
            clf.fit(X_train_resampled, y_train_resampled)

            # Prédictions sur TEST uniquement (jamais sur train !)
            y_test_pred = clf.predict(X_test_processed)

            # Métriques sur TEST
            metrics["recall_test"].append(
                recall_score(y_test_fold, y_test_pred, pos_label=pos_label)
            )
            metrics["precision_test"].append(
                precision_score(
                    y_test_fold, y_test_pred, pos_label=pos_label, zero_division=0
                )
            )
            metrics["f1_test"].append(
                f1_score(y_test_fold, y_test_pred, pos_label=pos_label)
            )

            if hasattr(clf, "predict_proba"):
                pos_idx = list(clf.classes_).index(pos_label)
                y_test_proba = clf.predict_proba(X_test_processed)[:, pos_idx]
                metrics["roc_auc_test"].append(
                    roc_auc_score((y_test_fold == pos_label).astype(int), y_test_proba)
                )

        results = {
            "model": model_name,
            "recall_test_mean": np.mean(metrics["recall_test"]),
            "recall_test_std": np.std(metrics["recall_test"]),
            "precision_test_mean": np.mean(metrics["precision_test"]),
            "precision_test_std": np.std(metrics["precision_test"]),
            "f1_test_mean": np.mean(metrics["f1_test"]),
            "f1_test_std": np.std(metrics["f1_test"]),
            "roc_auc_test_mean": np.mean(metrics["roc_auc_test"])
            if metrics["roc_auc_test"]
            else 0,
        }

        # Affichage TEST uniquement (pas de train avec resampling)
        print(f"\n{'=' * 60}")
        print(f" {model_name}")
        print(f"{'=' * 60}")
        print(f"\n{'Métrique':<15} {'Test (CV)':>20}")
        print("-" * 35)
        print(
            f"{'Recall':<15} {results['recall_test_mean']:.3f} ± {results['recall_test_std']:.3f}"
        )
        print(
            f"{'Precision':<15} {results['precision_test_mean']:.3f} ± {results['precision_test_std']:.3f}"
        )
        print(
            f"{'F1-Score':<15} {results['f1_test_mean']:.3f} ± {results['f1_test_std']:.3f}"
        )
        print(f"{'ROC-AUC':<15} {results['roc_auc_test_mean']:.3f}")

    return results


print("Fonction evaluate_model_cv() définie (avec support optionnel du resampling)")

## 12. Modèle Dummy (Baseline naïf)

Le `DummyClassifier` sert de **référence minimale**. Si nos vrais modèles ne font pas mieux, c'est qu'ils n'apprennent rien.

**Stratégie utilisée :** `most_frequent` → prédit toujours la classe majoritaire (0 = resté)


In [ ]:
from sklearn.dummy import DummyClassifier

# DummyClassifier avec validation croisée (comme référence)
print("🔄 Évaluation DummyClassifier avec validation croisée...")

dummy_clf = DummyClassifier(strategy="stratified", random_state=42)

# Utilisation de la fonction evaluate_model_cv définie plus haut
results_dummy = evaluate_model_cv(
    dummy_clf, X, y, preprocessor, model_name="DummyClassifier (stratified)"
)

print("\n⚠️ Ce modèle sert de BASELINE - tout modèle doit faire mieux !")

In [ ]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression sans class_weight (baseline)
print(
    "🔄 Évaluation Logistic Regression (sans class_weight) avec validation croisée..."
)

lr_clf = LogisticRegression(random_state=42, max_iter=1000)

results_lr = evaluate_model_cv(
    lr_clf, X, y, preprocessor, model_name="Logistic Regression (sans class_weight)"
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest sans class_weight (baseline)
print("🔄 Évaluation Random Forest (sans class_weight) avec validation croisée...")

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

results_rf = evaluate_model_cv(
    rf_clf, X, y, preprocessor, model_name="Random Forest (sans class_weight)"
)

## 13. Comparaison des Modèles Baseline

Récapitulatif des performances des 3 modèles **sans gestion du déséquilibre**.


In [ ]:
# Comparaison des modèles baseline (résultats de cross-validation)
baseline_results = [results_dummy, results_lr, results_rf]

comparison_df = pd.DataFrame(baseline_results)

# Sélection des colonnes principales
display_cols = [
    "model",
    "recall_test_mean",
    "recall_test_std",
    "precision_test_mean",
    "f1_test_mean",
    "roc_auc_test_mean",
]

comparison_display = comparison_df[display_cols].copy()
comparison_display.columns = [
    "Modèle",
    "Recall (CV)",
    "Recall Std",
    "Precision (CV)",
    "F1 (CV)",
    "ROC-AUC (CV)",
]

# Formatage
comparison_display["Recall (CV)"] = comparison_display.apply(
    lambda x: f"{x['Recall (CV)']:.3f} ± {x['Recall Std']:.3f}", axis=1
)
comparison_display = comparison_display.drop("Recall Std", axis=1)

print("COMPARAISON DES MODÈLES BASELINE (sans gestion du déséquilibre)")
print("\nMétriques en validation croisée (5 folds) - Focus classe 'Parti'\n")

display(comparison_display)

print("CONSTAT IMPORTANT :")
print("\n• L'Accuracy est élevée (~84%) mais TROMPEUSE !")
print("  → Un modèle qui prédit TOUJOURS 'Resté' aurait aussi ~84% d'accuracy")
print("\n• Le Recall est CATASTROPHIQUE (proche de 0)")
print("  → Les modèles ratent presque TOUS les départs")
print("\n• Conclusion : Il FAUT gérer le déséquilibre des classes !")

In [ ]:
# Visualisation comparative des modèles baseline

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Préparation des données à partir des résultats de cross-validation
models = ["Dummy", "Logistic Reg.", "Random Forest"]
recall_test = [
    results_dummy["recall_test_mean"],
    results_lr["recall_test_mean"],
    results_rf["recall_test_mean"],
]
precision_test = [
    results_dummy["precision_test_mean"],
    results_lr["precision_test_mean"],
    results_rf["precision_test_mean"],
]
f1_test = [
    results_dummy["f1_test_mean"],
    results_lr["f1_test_mean"],
    results_rf["f1_test_mean"],
]
recall_train = [
    results_dummy["recall_train_mean"],
    results_lr["recall_train_mean"],
    results_rf["recall_train_mean"],
]

# Graphique 1 : Métriques sur la classe 'Parti' (Test)
x = np.arange(len(models))
width = 0.25

ax1 = axes[0]
ax1.bar(x - width, precision_test, width, label="Precision", color="steelblue")
ax1.bar(x, recall_test, width, label="Recall", color="darkorange")
ax1.bar(x + width, f1_test, width, label="F1-Score", color="green")
ax1.set_ylabel("Score")
ax1.set_title("Métriques sur la classe 'Parti' (Test CV)")
ax1.set_xticks(x)
ax1.set_xticklabels(models)
ax1.legend()
ax1.set_ylim(0, 1)
ax1.axhline(y=0.5, color="red", linestyle="--", alpha=0.5, label="Seuil 50%")

# Graphique 2 : Comparaison Train vs Test (détection overfitting)
ax2 = axes[1]
x2 = np.arange(len(models))
ax2.bar(x2 - 0.2, recall_train, 0.4, label="Train", color="steelblue")
ax2.bar(x2 + 0.2, recall_test, 0.4, label="Test", color="darkorange")
ax2.set_ylabel("Recall (Parti)")
ax2.set_title("Détection Overfitting : Train vs Test")
ax2.set_xticks(x2)
ax2.set_xticklabels(models)
ax2.legend()
ax2.set_ylim(0, 1.1)

# Annotations pour Random Forest (overfitting)
ax2.annotate(
    "Overfitting!",
    xy=(2, recall_train[2]),
    xytext=(2, recall_train[2] + 0.05),
    ha="center",
    fontsize=10,
    color="red",
    fontweight="bold",
)

plt.tight_layout()
plt.show()

### 13.1 Analyse des résultats Baseline

**Constats observés :**

| Modèle                  | Comportement observé                              |
| ----------------------- | ------------------------------------------------- |
| **Dummy**               | Recall ~14% (prédiction aléatoire stratifiée)     |
| **Logistic Regression** | Recall ~40%, meilleur F1 parmi les baselines      |
| **Random Forest**       | Recall train=100% vs test=16% (**overfitting !**) |

**Problème identifié :** Sans gestion du déséquilibre (16% vs 84%), les modèles tendent à ignorer la classe minoritaire "Parti" ou à surapprendre sur le train set.


---

## Synthèse Partie 3 : Modélisation Baseline

### Modèles entraînés

| Modèle                   | Type                  | Objectif                     |
| ------------------------ | --------------------- | ---------------------------- |
| `DummyClassifier`        | Baseline naïf         | Référence minimale           |
| `LogisticRegression`     | Linéaire              | Premier modèle interprétable |
| `RandomForestClassifier` | Non-linéaire (arbres) | Capture relations complexes  |

### Métriques calculées

- ✅ `classification_report()` (Precision, Recall, F1)
- ✅ Matrice de confusion (Train ET Test)
- ✅ Comparaison Train vs Test pour détecter l'overfitting

### Problème identifié

**Déséquilibre des classes (16% Parti / 84% Resté)** :

- Les modèles optimisent l'accuracy → ignorent la classe minoritaire
- Le Recall sur "Parti" est insuffisant pour un usage métier RH


# Partie 4 : Gestion du Déséquilibre des Classes

---

## 14. Stratégie de gestion du déséquilibre

### 14.1 Contexte métier

Dans un contexte RH, **identifier les employés à risque de départ** (classe "Parti") est crucial :

- Un **Faux Négatif** (employé à risque non détecté) = départ non anticipé → coût élevé
- Un **Faux Positif** (employé stable mal classé) = actions RH inutiles → coût modéré

**Objectif** : Maximiser le **Recall** sur la classe "Parti" tout en maintenant une Precision acceptable.

### 14.2 Techniques à tester

| #   | Technique                 | Package  | Approche                     |
| --- | ------------------------- | -------- | ---------------------------- |
| 1   | `class_weight='balanced'` | sklearn  | Pondération des erreurs      |
| 2   | SMOTE                     | imblearn | Oversampling synthétique     |
| 3   | Random Undersampling      | imblearn | Réduction classe majoritaire |

### 14.3 Protocole d'évaluation

- **Validation croisée stratifiée** : `StratifiedKFold` (5 folds)
- **Métriques principales** : Recall, Precision, F1-score (classe "Parti")
- **Métrique secondaire** : ROC-AUC, PR-AUC


In [ ]:
from sklearn.pipeline import Pipeline as SkPipeline
from sklearn.metrics import precision_recall_curve, average_precision_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

print("Distribution des classes :")
print(y.value_counts(normalize=True).round(3))

## 15. Technique 1 : Class Weight (Pondération des classes)

### 15.1 Principe

Le paramètre `class_weight='balanced'` ajuste automatiquement les poids des classes inversement proportionnels à leur fréquence :

$$w_c = \frac{n_{samples}}{n_{classes} \times n_{samples_c}}$$

Cela pénalise davantage les erreurs sur la classe minoritaire ("Parti").


In [ ]:
# 18.2 Entraînement des modèles avec class_weight='balanced'
all_results = []  # Liste pour stocker tous les résultats

# Logistic Regression
print("🔄 Entraînement Logistic Regression avec class_weight='balanced'...")
lr_balanced = LogisticRegression(
    class_weight="balanced", max_iter=1000, random_state=42
)
results_lr_balanced = evaluate_model_cv(
    lr_balanced, X, y, preprocessor, model_name="Logistic Regression (balanced)"
)
all_results.append(results_lr_balanced)

# Random Forest
print("\n🔄 Entraînement Random Forest avec class_weight='balanced'...")
rf_balanced = RandomForestClassifier(
    n_estimators=100, class_weight="balanced", random_state=42, n_jobs=-1
)
results_rf_balanced = evaluate_model_cv(
    rf_balanced, X, y, preprocessor, model_name="Random Forest (balanced)"
)
all_results.append(results_rf_balanced)

## 16. Technique 2 : SMOTE (Oversampling)

### 16.1 Principe

**SMOTE** (Synthetic Minority Over-sampling Technique) génère des observations synthétiques pour la classe minoritaire en interpolant entre les observations existantes et leurs k plus proches voisins.

**Important** : SMOTE doit être appliqué **uniquement sur le jeu d'entraînement** pour éviter la fuite de données (data leakage).

**Note** : On utilise la même fonction `evaluate_model_cv()` avec le paramètre optionnel `resampler` pour éviter la duplication de code.


In [ ]:
# 17.2 Random Forest avec SMOTE
print("🔄 Entraînement Random Forest avec SMOTE...")

smote = SMOTE(random_state=42)

rf_smote = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# On utilise evaluate_model_cv avec le paramètre resampler
results_rf_smote = evaluate_model_cv(
    rf_smote, X, y, preprocessor, resampler=smote, model_name="Random Forest + SMOTE"
)
all_results.append(results_rf_smote)

## 17. Technique 3 : Undersampling

### 17.1 Principe

L'**undersampling** réduit le nombre d'observations de la classe majoritaire ("Resté") pour équilibrer les classes.

⚠️ **Inconvénient** : Perte d'information en supprimant des données.


In [ ]:
# 18.2 Random Forest avec Undersampling
print("Entraînement Random Forest avec Undersampling...")

undersampler = RandomUnderSampler(random_state=42)

rf_under = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# On utilise evaluate_model_cv avec le paramètre resampler
results_rf_under = evaluate_model_cv(
    rf_under,
    X,
    y,
    preprocessor,
    resampler=undersampler,
    model_name="Random Forest + Undersampling",
)
all_results.append(results_rf_under)

---

## 18. Comparaison des techniques de gestion du déséquilibre

### 18.1 Tableau récapitulatif

### 18.2 Comment lire les graphiques

**Graphique 1 - Trade-off Recall vs Precision :**

- **Axe X** : Recall (capacité à détecter les départs)
- **Axe Y** : Precision (fiabilité des alertes)
- **Lecture** : Un point en haut à droite = modèle idéal (bon recall ET bonne precision)
- **Objectif métier** : Privilégier le recall (détecter un maximum de départs)

**Graphique 2 - F1-Score par technique :**

- **Barres horizontales** : Score F1 (moyenne harmonique recall/precision)
- **Barres d'erreur** : Écart-type sur les 5 folds de validation croisée
- **Lecture** : Plus la barre est longue, meilleur est le compromis recall/precision


In [ ]:
# Création du DataFrame de comparaison
comparison_imbalance_df = pd.DataFrame(all_results)

# Sélection et formatage des colonnes principales
display_cols = [
    "model",
    "recall_test_mean",
    "recall_test_std",
    "precision_test_mean",
    "f1_test_mean",
    "roc_auc_test_mean",
]

comparison_display = comparison_imbalance_df[display_cols].copy()
comparison_display.columns = [
    "Modèle",
    "Recall (Test)",
    "Recall Std",
    "Precision (Test)",
    "F1 (Test)",
    "ROC-AUC (Test)",
]

# Formatage pour affichage
comparison_display["Recall (Test)"] = comparison_display.apply(
    lambda x: f"{x['Recall (Test)']:.3f} ± {x['Recall Std']:.3f}", axis=1
)
comparison_display = comparison_display.drop("Recall Std", axis=1)

print("COMPARAISON DES TECHNIQUES DE GESTION DU DÉSÉQUILIBRE")
print("\nFocus sur la classe 'Parti' (classe minoritaire)\n")

display(
    comparison_display.style.highlight_max(
        subset=["F1 (Test)", "ROC-AUC (Test)"], color="green"
    )
)

In [ ]:
# 22.2 Visualisation graphique
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1 : Recall vs Precision (Trade-off)
ax1 = axes[0]
models = comparison_imbalance_df["model"].values
recalls = comparison_imbalance_df["recall_test_mean"].values
precisions = comparison_imbalance_df["precision_test_mean"].values

colors_plot = plt.cm.Set2(np.linspace(0, 1, len(models)))
for i, (model, recall, precision) in enumerate(zip(models, recalls, precisions)):
    ax1.scatter(
        recall,
        precision,
        s=200,
        c=[colors_plot[i]],
        label=model,
        edgecolors="black",
        linewidth=1.5,
    )

ax1.set_xlabel("Recall (Test)", fontsize=12)
ax1.set_ylabel("Precision (Test)", fontsize=12)
ax1.set_title("Trade-off Recall vs Precision", fontsize=14, fontweight="bold")
ax1.legend(loc="best", fontsize=8)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)

# Graphique 2 : F1-Score par modèle
ax2 = axes[1]
f1_scores = comparison_imbalance_df["f1_test_mean"].values
f1_stds = comparison_imbalance_df["f1_test_std"].values

bars = ax2.barh(
    range(len(models)), f1_scores, xerr=f1_stds, color=colors_plot, edgecolor="black"
)
ax2.set_yticks(range(len(models)))
ax2.set_yticklabels(
    [m.replace(" + ", "\n+ ").replace(" (", "\n(") for m in models], fontsize=9
)
ax2.set_xlabel("F1-Score (Test)", fontsize=12)
ax2.set_title("F1-Score par technique", fontsize=14, fontweight="bold")
ax2.grid(True, alpha=0.3, axis="x")

# Ajout des valeurs sur les barres
for i, (bar, f1) in enumerate(zip(bars, f1_scores)):
    ax2.text(
        f1 + 0.02,
        bar.get_y() + bar.get_height() / 2,
        f"{f1:.3f}",
        va="center",
        fontsize=10,
    )

plt.tight_layout()
plt.show()

print("\nVisualisation des performances terminée")

### 18.3 Analyse des résultats

| Technique                   | Avantages                                   | Inconvénients                     |
| --------------------------- | ------------------------------------------- | --------------------------------- |
| **class_weight='balanced'** | Simple, pas de preprocessing supplémentaire | Peut ne pas suffire seul          |
| **SMOTE**                   | Génère des données synthétiques             | Risque d'overfitting, coût calcul |
| **Undersampling**           | Réduit le temps d'entraînement              | Perte d'information               |

#### Observations :

1. **Recall** : Les techniques de gestion du déséquilibre améliorent significativement le recall sur "Parti"
2. **Trade-off** : Amélioration du recall souvent au détriment de la precision
3. **F1-Score** : Compromis entre recall et precision


---

## 19. Courbe Precision-Recall

### 19.1 Principe

La courbe Precision-Recall permet de visualiser le trade-off entre la détection des départs (Recall) et la fiabilité des alertes (Precision).

### 19.2 Comment lire les graphiques

**Graphique 1 - Courbe Precision-Recall :**

- **Axe X** : Recall (proportion des vrais départs détectés)
- **Axe Y** : Precision (proportion des alertes qui sont de vrais départs)
- **Aire sous la courbe (AP)** : Plus elle est grande, meilleur est le modèle
- **Étoile rouge** : Point où le F1-Score est maximal

**Graphique 2 - F1-Score en fonction du seuil :**

- **Axe X** : Seuil de décision (entre 0 et 1)
- **Axe Y** : F1-Score correspondant
- **Ligne orange** : Seuil par défaut (0.5)
- **Note** : On garde le seuil 0.5 pour éviter l'optimisation excessive

**Graphique 3 - Courbe de Calibration :**

- **Ligne pointillée** : Calibration parfaite (si proba=0.3, 30% sont vraiment positifs)
- **Courbe bleue** : Calibration du modèle
- **Proche de la diagonale** = bonnes probabilités

**Graphique 4 - Distribution des probabilités :**

- **Vert** : Restés | **Rouge** : Partis
- **Bonne séparation** = les deux distributions sont bien distinctes
- **Chevauchement** = zone d'incertitude du modèle


In [ ]:
# Random Forest avec class_weight='balanced' pour analyse des probabilités
print("Entraînement RF balanced pour analyse des probabilités...")

rf_pipeline = SkPipeline(
    [
        ("preprocessor", preprocessor),
        (
            "classifier",
            RandomForestClassifier(
                n_estimators=100, class_weight="balanced", random_state=42, n_jobs=-1
            ),
        ),
    ]
)

rf_pipeline.fit(X_train, y_train)

# Probabilités sur le test set
y_test_proba = rf_pipeline.predict_proba(X_test)
parti_idx = list(rf_pipeline.classes_).index(1)
y_test_proba_parti = y_test_proba[:, parti_idx]
y_test_binary = y_test.values

print(f"✅ RF entraîné - Classes: {rf_pipeline.classes_}")

# Analyse des déciles
print("\nANALYSE DES DÉCILES")
deciles = np.percentile(y_test_proba_parti, np.arange(0, 101, 10))
for i, (low, high) in enumerate(zip(deciles[:-1], deciles[1:])):
    mask = (y_test_proba_parti >= low) & (y_test_proba_parti < high)
    if i == 9:
        mask = (y_test_proba_parti >= low) & (y_test_proba_parti <= high)
    n_obs = mask.sum()
    n_partis = y_test_binary[mask].sum() if n_obs > 0 else 0
    taux = (n_partis / n_obs * 100) if n_obs > 0 else 0
    print(
        f"   D{i + 1:2d} [{low:.3f}-{high:.3f}]: {n_obs:3d} obs, {n_partis:2.0f} départs ({taux:5.1f}%)"
    )

# Focus extrêmes
top_10 = np.percentile(y_test_proba_parti, 90)
bottom_10 = np.percentile(y_test_proba_parti, 10)
print(
    f"\nTop 10% (>= {top_10:.3f}): {(y_test_proba_parti >= top_10).sum()} empl, "
    f"{y_test_binary[y_test_proba_parti >= top_10].sum():.0f} départs"
)
print(
    f"Bottom 10% (<= {bottom_10:.3f}): {(y_test_proba_parti <= bottom_10).sum()} empl, "
    f"{y_test_binary[y_test_proba_parti <= bottom_10].sum():.0f} départs"
)

In [ ]:
# 23.3 Courbe Precision-Recall et Courbe de Calibration
from sklearn.calibration import calibration_curve

# Courbe Precision-Recall
precision_curve, recall_curve, thresholds = precision_recall_curve(
    y_test_binary, y_test_proba_parti
)
ap_score = average_precision_score(y_test_binary, y_test_proba_parti)

# Calcul du F1-score pour chaque seuil
f1_scores = (
    2
    * (precision_curve[:-1] * recall_curve[:-1])
    / (precision_curve[:-1] + recall_curve[:-1] + 1e-10)
)
best_threshold_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_idx]
best_f1 = f1_scores[best_threshold_idx]
best_precision = precision_curve[best_threshold_idx]
best_recall = recall_curve[best_threshold_idx]


fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Graphique 1 : Courbe Precision-Recall
ax1 = axes[0, 0]
ax1.plot(
    recall_curve,
    precision_curve,
    "b-",
    linewidth=2,
    label=f"PR Curve (AP = {ap_score:.3f})",
)
ax1.scatter(
    [best_recall],
    [best_precision],
    s=200,
    c="red",
    marker="*",
    zorder=5,
    label=f"Max F1 @ seuil {best_threshold:.3f}",
)
ax1.axhline(
    y=y_test_binary.mean(), color="gray", linestyle="--", label="Baseline (random)"
)
ax1.set_xlabel("Recall", fontsize=12)
ax1.set_ylabel("Precision", fontsize=12)
ax1.set_title("Courbe Precision-Recall", fontsize=14, fontweight="bold")
ax1.legend(loc="best")
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)

# Graphique 2 : F1-Score en fonction du seuil
ax2 = axes[0, 1]
ax2.plot(thresholds, f1_scores, "g-", linewidth=2)
ax2.axvline(
    x=best_threshold,
    color="red",
    linestyle="--",
    label=f"Max F1 = {best_f1:.3f}",
)
ax2.scatter([best_threshold], [best_f1], s=200, c="red", marker="*", zorder=5)
ax2.axvline(x=0.5, color="orange", linestyle=":", label="Seuil par défaut (0.5)")
ax2.set_xlabel("Seuil de décision", fontsize=12)
ax2.set_ylabel("F1-Score", fontsize=12)
ax2.set_title("F1-Score en fonction du seuil", fontsize=14, fontweight="bold")
ax2.legend(loc="best")
ax2.grid(True, alpha=0.3)

# Graphique 3 : COURBE DE CALIBRATION
ax3 = axes[1, 0]
prob_true, prob_pred = calibration_curve(y_test_binary, y_test_proba_parti, n_bins=10)
ax3.plot([0, 1], [0, 1], "k--", label="Calibration parfaite")
ax3.plot(prob_pred, prob_true, "s-", color="blue", label="Random Forest")
ax3.set_xlabel("Probabilité moyenne prédite", fontsize=12)
ax3.set_ylabel("Fraction de positifs (réelle)", fontsize=12)
ax3.set_title("Courbe de Calibration", fontsize=14, fontweight="bold")
ax3.legend(loc="best")
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0, 1)
ax3.set_ylim(0, 1)

# Graphique 4 : Distribution des probabilités
ax4 = axes[1, 1]
ax4.hist(
    y_test_proba_parti[y_test_binary == 0],
    bins=30,
    alpha=0.5,
    label="Restés",
    color="green",
)
ax4.hist(
    y_test_proba_parti[y_test_binary == 1],
    bins=30,
    alpha=0.5,
    label="Partis",
    color="red",
)
ax4.axvline(x=0.5, color="orange", linestyle=":", linewidth=2, label="Seuil 0.5")
ax4.set_xlabel("Probabilité de départ", fontsize=12)
ax4.set_ylabel("Nombre d'employés", fontsize=12)
ax4.set_title(
    "Distribution des probabilités par classe", fontsize=14, fontweight="bold"
)
ax4.legend(loc="best")
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Métriques avec seuil par défaut (0.5)
print("MÉTRIQUES AVEC SEUIL PAR DÉFAUT (0.5)")
y_pred_default = (y_test_proba_parti >= 0.5).astype(int)
print(f"   Recall:    {recall_score(y_test_binary, y_pred_default):.3f}")
print(
    f"   Precision: {precision_score(y_test_binary, y_pred_default, zero_division=0):.3f}"
)
print(f"   F1-Score:  {f1_score(y_test_binary, y_pred_default):.3f}")

In [ ]:
# Matrice de confusion (seuil = 0.5)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_test_pred_default = y_pred_default  # Réutilisation

fig, ax = plt.subplots(1, 1, figsize=(7, 5))
cm_default = confusion_matrix(y_test, y_test_pred_default, labels=[0, 1])
ConfusionMatrixDisplay(cm_default, display_labels=["Resté", "Parti"]).plot(
    ax=ax, cmap="Blues", values_format="d"
)
ax.set_title("Matrice de Confusion (seuil = 0.5)", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

# Métriques
tn, fp, fn, tp = cm_default.ravel()
recall_val = tp / (tp + fn) if (tp + fn) > 0 else 0
precision_val = tp / (tp + fp) if (tp + fp) > 0 else 0
f1_val = (
    2 * precision_val * recall_val / (precision_val + recall_val)
    if (precision_val + recall_val) > 0
    else 0
)

print(f"TN: {tn} | FP: {fp} | FN: {fn} | TP: {tp}")
print(f"Recall: {recall_val:.3f} | Precision: {precision_val:.3f} | F1: {f1_val:.3f}")

---

## Synthèse Partie 4 : Gestion du déséquilibre des classes

### Résultats comparatifs des techniques

| Technique                          | Recall    | Precision | F1-Score  | ROC-AUC   |
| ---------------------------------- | --------- | --------- | --------- | --------- |
| **Logistic Regression (balanced)** | **0.743** | 0.378     | **0.501** | **0.830** |
| Random Forest (balanced)           | 0.139     | 0.872     | 0.235     | 0.807     |
| Random Forest + SMOTE              | 0.262     | 0.735     | 0.381     | 0.829     |
| Random Forest + Undersampling      | 0.700     | 0.369     | 0.482     | 0.813     |

**Meilleur modèle Partie 4** : **Logistic Regression avec class_weight='balanced'** (Recall=74.3%, F1=0.501, ROC-AUC=0.830)

### Conclusions

1. **Logistic Regression > Random Forest** pour ce problème (meilleur Recall)
2. **class_weight='balanced'** est la technique la plus simple et efficace
3. **Undersampling** donne un excellent Recall (0.700) mais perd en Precision

➡️ **Partie 5** : Optimisation avec LightGBM pour améliorer les performances.


---

# Partie 5 : Fine-tuning et Interprétabilité (SHAP)

## Objectifs

1. **Fine-tuning** : Optimiser les hyperparamètres avec GridSearchCV (CV stratifiée)
2. **Modèle final** : LightGBM avec forte régularisation pour éviter l'overfitting
3. **SHAP** : Interprétabilité globale (Beeswarm) et locale (Waterfall)

### Pourquoi SHAP ?

SHAP (SHapley Additive exPlanations) est basé sur la **théorie des jeux** de Lloyd Shapley (Prix Nobel d'économie 2012).

- **Mathématiquement juste** : Seule méthode satisfaisant toutes les propriétés d'équité
- **Interprétable** : Comprendre l'impact de chaque feature sur la prédiction


---

## 20. Imports et préparation Partie 5


In [ ]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import shap
import warnings

warnings.filterwarnings("ignore")
print("Imports Partie 5 chargés (GridSearchCV, LightGBM, SHAP)")

In [ ]:
# GridSearchCV sur LightGBM avec forte régularisation
print("GRIDSEARCHCV - LightGBM avec régularisation")

pipeline_lgb = SkPipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", lgb.LGBMClassifier(random_state=42, verbose=-1, n_jobs=-1)),
    ]
)

# Grille avec forte régularisation pour éviter l'overfitting
param_grid = {
    "classifier__n_estimators": [30, 50, 75],
    "classifier__max_depth": [2, 3],
    "classifier__learning_rate": [0.01, 0.03, 0.05],
    "classifier__class_weight": ["balanced"],
    "classifier__num_leaves": [4, 7, 10],
    "classifier__min_child_samples": [50, 75, 100],
    "classifier__reg_alpha": [1.0, 5.0, 10.0],
    "classifier__reg_lambda": [1.0, 5.0, 10.0],
    "classifier__subsample": [0.7, 0.8],
    "classifier__colsample_bytree": [0.7, 0.8],
}

f1_scorer = make_scorer(f1_score, pos_label=1)

grid_search = GridSearchCV(
    pipeline_lgb,
    param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring=f1_scorer,
    n_jobs=-1,
    verbose=1,
    refit=True,
)

grid_search.fit(X_train, y_train)

print("\nMeilleurs paramètres:")
for param, value in grid_search.best_params_.items():
    print(f"   {param}: {value}")
print(f"\nMeilleur F1 (CV): {grid_search.best_score_:.3f}")

In [ ]:
# Évaluation du meilleur modèle sur le test set
best_model = grid_search.best_estimator_
y_proba_optimized = best_model.predict_proba(X_test)[:, 1]
y_pred_optimized = (y_proba_optimized >= 0.5).astype(int)

print("MÉTRIQUES SUR LE JEU DE TEST:")
print(f"   Recall:     {recall_score(y_test, y_pred_optimized, pos_label=1):.3f}")
print(
    f"   Precision:  {precision_score(y_test, y_pred_optimized, pos_label=1, zero_division=0):.3f}"
)
print(f"   F1-Score:   {f1_score(y_test, y_pred_optimized, pos_label=1):.3f}")
print(f"   ROC-AUC:    {roc_auc_score(y_test, y_proba_optimized):.3f}")

---

## 21. Interprétabilité avec SHAP

### Pourquoi SHAP ?

SHAP (SHapley Additive exPlanations) est basé sur les **valeurs de Shapley** de la théorie des jeux :

- **Mathématiquement juste** : Seule méthode qui respecte toutes les propriétés d'équité (efficacité, symétrie, linéarité, nullité)
- **Interprétable** : Chaque feature reçoit une "contribution" à la prédiction
- **Global et local** : Permet de comprendre le modèle dans son ensemble ET chaque prédiction individuelle

### Types d'analyses

1. **Beeswarm Plot** (global) : Vue d'ensemble de l'importance de chaque feature
2. **Permutation Importance** (global) : Comparaison avec méthode sklearn
3. **Waterfall Plot** (local) : Explication d'une prédiction individuelle


In [ ]:
# Récupérer le preprocessor et le classifier du meilleur modèle
preprocessor_fitted = best_model.named_steps["preprocessor"]
classifier_fitted = best_model.named_steps["classifier"]

# Transformer les données de test
X_test_transformed = preprocessor_fitted.transform(X_test)

# Récupérer les noms des features après transformation
# Note: get_feature_names_out() sans argument utilise les noms appris lors du fit
cat_feature_names = (
    preprocessor_fitted.named_transformers_["cat"].get_feature_names_out().tolist()
)
num_feature_names = (
    num_cols.copy()
)  # num_cols est définie dans le preprocessing (23 colonnes)
all_feature_names = num_feature_names + cat_feature_names

print(f"Nombre de features après transformation: {len(all_feature_names)}")
print(f"   - Features numériques: {len(num_feature_names)}")
print(f"   - Features catégorielles (après OHE): {len(cat_feature_names)}")

In [ ]:
print("Calcul des SHAP values (TreeExplainer pour LightGBM)...")

# TreeExplainer est optimisé pour les modèles à base d'arbres
explainer = shap.TreeExplainer(classifier_fitted)
shap_values = explainer.shap_values(X_test_transformed)

# Pour classification binaire, shap_values peut être une liste [classe_0, classe_1]
if isinstance(shap_values, list):
    shap_values = shap_values[1]  # On prend la classe positive (Parti)

print("SHAP values calculées")
print(f"   Shape: {shap_values.shape}")

### 21.1 Feature Importance Globale - Beeswarm Plot

Le **Beeswarm Plot** montre :

- **Axe Y** : Les features triées par importance
- **Axe X** : L'impact sur la prédiction (SHAP value)
- **Couleur** : La valeur de la feature (rouge = élevée, bleu = basse)


In [ ]:
# SHAP Beeswarm Plot - Importance globale des features
X_test_df = pd.DataFrame(X_test_transformed, columns=all_feature_names)

plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_test_df, plot_type="dot", show=False, max_display=20)
plt.title(
    "SHAP - Impact des features sur la probabilité de départ",
    fontsize=12,
    fontweight="bold",
)
plt.tight_layout()
plt.show()

### 21.2 Feature Importance Locale - Waterfall Plot

Le **Waterfall Plot** explique UNE prédiction individuelle :

- Comment chaque feature a contribué à passer de la valeur de base (moyenne) à la prédiction finale
- Utile pour expliquer une décision à un manager RH


In [ ]:
# Préparation des exemples pour waterfall plots
y_pred_test = y_pred_optimized
true_positives = np.where((y_test == 1) & (y_pred_test == 1))[0]
true_negatives = np.where((y_test == 0) & (y_pred_test == 0))[0]

print(f"Vrais Positifs: {len(true_positives)} | Vrais Négatifs: {len(true_negatives)}")

# Objet Explanation pour waterfall
explanation = shap.Explanation(
    values=shap_values,
    base_values=np.full(
        len(shap_values),
        explainer.expected_value
        if not hasattr(explainer.expected_value, "__len__")
        else explainer.expected_value[1],
    ),
    data=X_test_transformed,
    feature_names=all_feature_names,
)

In [ ]:
# Waterfall Plot - Exemple d'un employé "Parti"
if len(true_positives) > 0:
    idx_parti = true_positives[0]
    proba_parti = y_proba_optimized[idx_parti]

    print(f"Employé PARTI - Index: {idx_parti}, Proba: {proba_parti:.3f}")

    plt.figure(figsize=(12, 8))
    shap.plots.waterfall(explanation[idx_parti], max_display=15, show=False)
    plt.title(
        f"Waterfall - Employé Parti (proba={proba_parti:.3f})",
        fontsize=12,
        fontweight="bold",
    )
    plt.tight_layout()
    plt.show()
else:
    print("Aucun vrai positif trouvé")

---

## Synthèse Partie 5 : Fine-tuning et Interprétabilité

### Modèle final : LightGBM avec GridSearchCV (régularisé)

| Métrique      | Valeur   | Interprétation                                      |
|---------------|----------|-----------------------------------------------------|
| **Recall**    | **0.596**| Détecte 28 départs sur 47 (59.6%)                   |
| **Precision** | 0.368    | 28 alertes correctes sur 76 levées                  |
| **F1-Score**  | **0.455**| Meilleur compromis recall/precision                 |
| **ROC-AUC**   | 0.800    | Bonne capacité de discrimination                    |

### 🔧 Meilleurs hyperparamètres (GridSearchCV)

```
learning_rate: 0.05       # Apprentissage lent → généralisation
max_depth: 3              # Arbres peu profonds → évite overfitting
n_estimators: 75          # Nombre d'arbres modéré
num_leaves: 10            # Peu de feuilles → modèle simple
min_child_samples: 50     # Minimum 50 samples/feuille → robustesse
reg_alpha: 1.0            # Régularisation L1
reg_lambda: 1.0           # Régularisation L2  
subsample: 0.7            # Bagging 70% → réduit variance
colsample_bytree: 0.7     # Feature sampling 70%
class_weight: balanced    # Gestion du déséquilibre
```

### ✅ Stratégie anti-overfitting

Le modèle utilise une **forte régularisation** pour éviter l'overfitting observé avec Random Forest :

| Technique | Effet |
|-----------|-------|
| `max_depth=3` | Arbres superficiels, pas de mémorisation |
| `reg_alpha/lambda=1.0` | Pénalise les poids extrêmes |
| `subsample=0.7` | Bagging pour réduire la variance |
| `min_child_samples=50` | Évite les feuilles avec peu d'exemples |

**Résultat** : Le F1 en cross-validation (0.534) est proche du F1 sur test (0.455), ce qui indique une bonne généralisation.

### SHAP - Top 5 Features les plus impactantes

1. **heures_supplementaires_Oui** - Faire des heures sup **augmente fortement** le risque de départ
2. **nombre_participation_pee** - Faible participation au PEE → risque accru
3. **satisfaction_globale** - Faible satisfaction globale → risque accru
4. **age** - Employés plus jeunes → risque accru de départ
5. **revenu_mensuel** - Un salaire **bas** augmente le risque de départ

### Exemple d'interprétation locale (SHAP Waterfall)

L'employé prédit comme "Parti" montre :
- **Heures supplémentaires** : facteur majeur d'augmentation du risque
- **Jeune âge** : contribue au risque
- **Nombreuses expériences précédentes** : signe d'instabilité
- **Participation au PEE** : facteur de rétention

### Recommandations métier pour TechNova Partners

| Priorité | Action | Impact attendu |
|----------|--------|----------------|
| 🔴 Haute | **Limiter les heures supplémentaires** récurrentes | Réduit le facteur #1 de départ |
| 🔴 Haute | **Réviser les grilles salariales** (employés sous-payés) | Améliore la rétention |
| 🟠 Moyenne | Proposer **télétravail/horaires flexibles** | Équilibre vie pro/perso |
| 🟠 Moyenne | **Promouvoir le PEE** (Plan Épargne Entreprise) | Engagement financier |
| 🟡 Basse | **Programmes de mentorat** pour les jeunes talents | Fidélisation des juniors |

### Conclusion générale

Le modèle **LightGBM régularisé** permet de :
- ✅ **Identifier** les employés à risque avec un **Recall de 59.6%** (détecte 28 départs sur 47)
- ✅ **Comprendre** les facteurs de départ grâce à **SHAP** (heures sup, PEE, satisfaction, âge)
- ✅ **Prioriser** les actions RH avec des **probabilités calibrées** (10% à 90%)
- ✅ **Généraliser** correctement grâce à la régularisation

### ⚠️ Limites et améliorations possibles

| Limite | Amélioration potentielle |
|--------|--------------------------|
| Recall de 60% = 40% de départs non détectés | Collecter plus de données |
| Dataset de 1176 employés | Ajouter données longitudinales |
| Features SIRH uniquement | Intégrer feedback entretiens, engagement teams |

---
